## Figure Friday 2024 - W48
This week’s Figure-Friday looks at the Worldbank’s data on Individuals using the Internet (as a % of the population). It’s important to note that internet users are defined as individuals who have used the Internet (from any location) in the last 3 months. (The Internet can be used via a computer, mobile phone, personal digital assistant, games machine, digital TV etc.)</br>
The challenge:
+ Improve the sample figure figure-friday-2024-week-48
+ would a different figure tell the data story better?
+ can you create a Dash app instead?

The **first** attempt showcases the top 5 and bottom 5 country growth related to the Internet penetration. It has a picker random number between 1990 and 2023, for which there are data-collected.
And the **second** version has an improved layout for the dashboard app that consider some other callbacks and charts.

In [2]:
"""Just importing modules"""
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import numpy as np
import random

np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.precision', 3)
pd.options.display.float_format = '{:,.3f}'.format # to suppress scientific notation

# df_arg = pd.read_csv(r"C:\Users\Juan Aguirre\Downloads\Book2.csv")
df = pd.read_csv("https://raw.githubusercontent.com/plotly/Figure-Friday/refs/heads/main/2024/week-48/API_IT.NET.USER.ZS_DS2_en_csv_v2_2160.csv")

exclude_cols = ['Country Code', 'Indicator Name', 'Indicator Code']
# df[df.columns.difference(exclude_cols, sort=False)]

df_melted = (pd.melt(
    df[df.columns.difference(exclude_cols, sort=False)],
    id_vars=['Country Name'],
    var_name='Year',
    value_name='Quantity'
))

df_melted['Year'] = pd.to_numeric(df_melted['Year'], errors='coerce')

# Drop rows where 'Year' is NaN (non-year columns) or 'Quantity' is NaN
df_melted = df_melted.dropna(subset=['Year', 'Quantity'])

# df_country_year_idxd = (df_melted.set_index(keys=['Country Name', 'Year']))
df_year_idxd = (df_melted.set_index(keys=['Year']))
# Initialize the Dash app with Bootstrap theme
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout definition with Bootstrap components
button = html.Div(
    [
        dbc.Button("Pick Random Year", id="submit-button", className="btn btn-primary me-2", n_clicks=0),
        html.Span(id="selected-year-output", style={"verticalAlign": "middle"}),
    ]
)

app.layout = dbc.Container(
    fluid=True,
    children=[
            dbc.Row([
                dbc.Col(
                    html.H3('Top and Bottom 5 countries by Internet Growth',
                            className="text-start text-primary mb-2"),
                ),
            ]),
            dbc.Row([
                dbc.Col(button,
                ),
            ]),
            dbc.Row([
                dbc.Col(dcc.Graph(id="top-5-graph", figure={}), width=6),
                dbc.Col(dcc.Graph(id="bottom-5-graph", figure={}), width=6)
            ]),
    ]
)

# Callback to handle random year selection and graph updates
@app.callback(
    Output("selected-year-output", "children"),
    Output("top-5-graph", "figure"),
    Output("bottom-5-graph", "figure"),
    Input("submit-button", "n_clicks")
)
def update_graphs(n_clicks):
    if n_clicks == 0:
        return "Click the button to pick a random year.", {}, {}

    # Randomly select a year between 1992 and 2023
    selected_year = random.randint(1992, 2023)

    # Filter data for the selected year
    filtered = df_year_idxd.loc[selected_year]

    # Get Top 5 countries by Quantity
    top_5 = filtered.nlargest(5, "Quantity")

    # Get Bottom 5 countries by Quantity
    bottom_5 = filtered.nsmallest(5, "Quantity")

    # Create Top 5 bar plot
    fig_top_5 = px.bar(top_5, y="Country Name", x="Quantity", title=f"Top 5 Countries in {selected_year}",
                       text_auto=True, color_discrete_sequence=["#FF7F0E"], orientation='h', template='plotly_white',
                       labels={'Country Name': '', 'Quantity': ''})

    # Create Bottom 5 bar plot
    fig_bottom_5 = px.bar(bottom_5, y="Country Name", x="Quantity", title=f"Bottom 5 Countries in {selected_year}",
                          text_auto=True, color_discrete_sequence=['#BCBD22'], orientation='h', template='plotly_white',
                          labels={'Country Name': '', 'Quantity': ''})

    return f"Selected Year: {selected_year}", fig_top_5, fig_bottom_5


if __name__ == "__main__":
    app.run(debug=True, jupyter_mode='inline')